# 금연구역 행정동 추출

In [ ]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import requests
import json

In [ ]:
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'


df = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_성공(44475).csv', encoding ='cp949')

## 데이터 확인

In [ ]:
df.head(2)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,대전광역시 중구 목중로26번길 83,83,중구 목중로26번길 중앙독서실,중촌동,36.338707,127.414436
1,중구,게임제공업소,지존PC방,목중로54번길 2,대전광역시 중구 목중로54번길 2 2층 (중촌동),2층,중구 목중로54번길 2 지존PC방,판암동,36.324284,127.458253


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44475 entries, 0 to 44474
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   구         44475 non-null  object 
 1   구분        44475 non-null  object 
 2   시설명       42771 non-null  object 
 3   주소        44473 non-null  object 
 4   번지주소      44475 non-null  object 
 5   세부주소      12215 non-null  object 
 6   kakao 주소  44474 non-null  object 
 7   동         44473 non-null  object 
 8   위도        44475 non-null  float64
 9   경도        44475 non-null  float64
dtypes: float64(2), object(8)
memory usage: 3.4+ MB


## kakao api를 통한 행정동 추출

1. 주소를 입력하여 행정동 추출
2. 위도 경도를 입력하여 행정동 추출

2번 시도

### kakao api 세팅

In [ ]:
# rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

api_key = "616734aa8214dbf958cfaebd6bd41aec"

In [ ]:
# 위경도를 입력받는 kakao api 함수
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][1]['region_3depth_name']
    return str(match_first)

In [ ]:
# 리스트에 담아줌
df_lati = df['위도']
df_long = df['경도']

In [ ]:
# 확인
df_lati, df_long

(0        36.338707
 1        36.324284
 2        36.367414
 3        36.335889
 4        36.367414
            ...    
 44470    36.322869
 44471    36.344371
 44472    36.349017
 44473    36.349428
 44474    36.349134
 Name: 위도, Length: 44475, dtype: float64,
 0        127.414436
 1        127.458253
 2        127.428685
 3        127.416505
 4        127.428685
             ...    
 44470    127.442877
 44471    127.443010
 44472    127.441479
 44473    127.438749
 44474    127.441477
 Name: 경도, Length: 44475, dtype: float64)

In [ ]:
add_list1 = df_long
add_list2 = df_lati

In [ ]:
add_list2[4], add_list1[4]

(36.3674143085976, 127.42868473455)

In [ ]:
# 함수 확인
lat_lon_to_addr(126.981204133005, 37.5555892070291)

'회현동'

In [ ]:
# 함수 확인
lat_lon_to_addr(127.3983633, 36.3136886)

'유천2동'

In [ ]:
df_dong = df.copy()

In [ ]:
# 행정동 컬럼 생성
df_dong['행정동'] = np.nan

In [ ]:
df_dong.head(0)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도,행정동


In [ ]:
# 행정동 추출 반복문
for i in tqdm(range(len(add_list1))):
    try :
        df_dong['행정동'].iloc[i] = lat_lon_to_addr(add_list1[i], add_list2[i])
    except:
        pass

print(df_dong.isnull().sum())

  0%|          | 0/44475 [00:00<?, ?it/s]C:\Users\JBY\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 44475/44475 [42:29<00:00, 17.45it/s]  

구               0
구분              0
시설명          1704
주소              2
번지주소            0
세부주소        32260
kakao 주소        1
동               2
위도              0
경도              0
행정동             0
dtype: int64


In [ ]:
df_dong.head(2)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도,행정동
0,중구,학교교과교습학원,중앙독서실,목중로26번길,대전광역시 중구 목중로26번길 83,83,중구 목중로26번길 중앙독서실,중촌동,36.338707,127.414436,중촌동
1,중구,게임제공업소,지존PC방,목중로54번길 2,대전광역시 중구 목중로54번길 2 2층 (중촌동),2층,중구 목중로54번길 2 지존PC방,판암동,36.324284,127.458253,판암2동


전체 추출 성공 

## 데이터프레임으로 저장

In [ ]:
df_dong.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_행정동추출(44475).csv", index=False, encoding='cp949')